### Purpose of this Jupyter Notebook:

#### Background:

- The goal is to use SWE as a indicator of water content in Indus basin, where the main source of water comes as a result of snow melt in the mountains.
- The aim is to use SWE as a index for mapping areas that may experience drought and to analyze how if affects food security in vulnerable communities

#### How and what we are trying to achieve from this
- Visualization of SWE (12 years data, 2011-2022)
- Use python libraries to extract the zonal statistics for each input layer
- Use the statistics for time series visualization using SWE

NOTE: The same script can be used to visualize other parameters available from FEWSNET

####  Required inputs:
- SHP file: shapefile containing your AOI
- Access to dataset files (local server path or online directory link)
    - In this notebook (file format: .nc files) - which is a NetCDF dataset
- Input the Parameter of Interest (Here, only SWE has been used)
- Input Year and Date
    - The datasets are in order of yearly basis from October of one year to September of next year
    - The datasets are separated by each year
- Input the Statistics metrics required for analysis for user's AOI: For eg: min, max, mean, count, etc

#### Output:
- Statistics files (.csv file containing the Basic Statistics for selected Parameter)
    - In this notebook, statistics metrics contain the Maximum, Minimum, Mean, Standard Deviation, Median values for each of AOI provided by the user
    - The user also may add other statistical metrics which maybe necessary accordinginly

### Importing Python libraries

In [44]:
# Importing python libraries

import glob
import xarray as xr
from datetime import datetime
import pandas as pd
import os
import numpy as np
import netCDF4 as nc

import rasterio as rio
from rasterio.plot import show
from rasterio.plot import show_hist

import rasterstats as rstats

import geopandas as gpd
import shapely
from shapely.geometry import mapping

import matplotlib.pyplot as plt
from matplotlib.pyplot import show

import dateutil
import fiona
from siphon.catalog import get_latest_access_url
from siphon.catalog import TDSCatalog

### Accessing the SHP file to be used to generate Statistics

In [45]:
#Input the SHP layer containing all the polygons who statistics are required for analysis

#Contains the SHP file for Indus Sub-Basins
shp_indus = gpd.read_file(r'D:\AOI\Indus_subbasins\Indus_sub.shp', crs = "epsg:4326")
shp_indus

,SHAPE_Area,L1_Name,L2_Name,L3_Name,Mcode,ID,excess_sno,geometry
0,437486.488345,Indus,Trunk Indus,Trunk Indus,None,45,NaN,"MULTIPOLYGON (((67.68004 23.80865, 67.67747 23..."
1,105740.483461,Indus,Panjnad,Sutlej,Ind17,47,17.31,"POLYGON ((81.22510 31.15371, 81.21941 31.15249..."
2,19504.518515,Indus,Panjnad,Beas,Ind16,48,NaN,"POLYGON ((76.03089 32.52178, 76.03457 32.52056..."
3,34859.405662,Indus,Panjnad,Trunk Panjnad,None,49,NaN,"POLYGON ((72.14429 31.17199, 72.14953 31.16611..."
4,30606.520298,Indus,Panjnad,Ravi,Ind15,50,5.25,"POLYGON ((77.07933 32.39681, 77.07442 32.39492..."
5,44856.317288,Indus,Panjnad,Chenab,Ind14,51,4.16,"POLYGON ((75.66970 34.19920, 75.67260 34.19920..."
6,15851.043097,Indus,Upper Indus,Zanskar,Ind10,52,NaN,"POLYGON ((78.16841 33.14484, 78.16350 33.14451..."
7,50858.832116,Indus,Panjnad,Jhelum,Ind13,53,10.74,"POLYGON ((73.99051 35.18002, 73.99386 35.17936..."
8,124888.298058,Indus,Trunk Indus,Trunk Indus,None,54,NaN,"POLYGON ((73.38567 34.07585, 73.39248 34.07108..."
9,17598.002492,Indus,Kabul,Kabul,None,56,10.28,"POLYGON ((70.38676 34.66044, 70.39055 34.65844..."


### Accessing the Fewsnet Dataset for each year from 2011 to 2022

- Due to some server connection errors, the local server (Option 2) was used to access the datasets from FEWSNET
- FILTERing and SORTing the NetCDF files only as some folders consists of other files as well

#### Option 01: Using TDS Thredds Catalogue

In [46]:
#Each variables contains the list of daily NetCDF dataset files (.nc files)
#Variable name syntax: fews_'year' eg: fews_2010 represents year 2010 data from FEWSNET

fews_2011 = 'http://110.34.30.197:8080/thredds/catalog/fewsnetData/1KM_NEW/WY2011/catalog.xml'
fews_2012 = 'http://110.34.30.197:8080/thredds/catalog/fewsnetData/1KM_NEW/WY2012/catalog.xml'
fews_2013 = 'http://110.34.30.197:8080/thredds/catalog/fewsnetData/1KM_NEW/WY2013/catalog.xml'
fews_2014 = 'http://110.34.30.197:8080/thredds/catalog/fewsnetData/1KM_NEW/WY2014/catalog.xml'
fews_2015 = 'http://110.34.30.197:8080/thredds/catalog/fewsnetData/1KM_NEW/WY2015/catalog.xml'
fews_2016 = 'http://110.34.30.197:8080/thredds/catalog/fewsnetData/1KM_NEW/WY2016/catalog.xml'
fews_2017 = 'http://110.34.30.197:8080/thredds/catalog/fewsnetData/1KM_NEW/WY2017/catalog.xml'
fews_2018 = 'http://110.34.30.197:8080/thredds/catalog/fewsnetData/1KM_NEW/WY2018/catalog.xml'
fews_2019 = 'http://110.34.30.197:8080/thredds/catalog/fewsnetData/1KM_NEW/WY2019/catalog.xml'
fews_2020 = 'http://110.34.30.197:8080/thredds/catalog/fewsnetData/1KM_NEW/WY2020/catalog.xml'
fews_2021 = 'http://110.34.30.197:8080/thredds/catalog/fewsnetData/1KM_NEW/WY2021/catalog.xml'
fews_2022 = 'http://110.34.30.197:8080/thredds/catalog/fewsnetData/1KM_NEW/WY2022/catalog.xml'

##### Getting list of all folders which contains the yearly NetCDF data

In [47]:
#Get list of all folders inside the FewsNET (indicating data from each year 2011-2022)
#URL to the list of folders
fews_dataset = 'http://110.34.30.197:8080/thredds/catalog/fewsnetData/1KM_NEW/.xml'

# NOTE: This list was extracted using the TDS catalog service
fews_direc = TDSCatalog(fews_dataset)
fews_wy_list = list(fews_direc.catalog_refs)
fews_wy_list.sort()

#EXAMPLE OF THE LIST: ['WY2011','WY2012','WY2013','WY2014','WY2015','WY2016','WY2017','WY2018','WY2019','WY2020','WY2021','WY2022']

#### Option 02: Using Local Server access

In [49]:
#Example of file path for the fewsNET dataset

file_path_2011 ='//192.168.10.74/fewsNET/1KM_NEW/WY2011'

##### Getting list of all folders which contains the yearly NetCDF data

In [22]:
#Get list of all folders inside the FewsNET (indicating data from each year 2011-2022)

# NOTE: This list was extracted using local access to server

folder_list_path = '//192.168.10.74/fewsNET/1KM_NEW'
folder_list = os.listdir(folder_list_path)

#EXAMPLE OF THE LIST: ['WY2011','WY2012','WY2013','WY2014','WY2015','WY2016','WY2017','WY2018','WY2019','WY2020','WY2021','WY2022']

#### USER INPUT REQUIRED:

##### ENTER the Year 

In [54]:
#USER INPUT REQUIRED:

############################################

#Enter the year for which you want the data:
year = 2022   #Assign Year

############################################

#Auto assign of year of the fewsDataset folder in the server
year_fews = "WY"+str(year)

#Auto assign file path
file_path = '//192.168.10.74/fewsNET/1KM_NEW/WY'+str(year)
file_path

'//192.168.10.74/fewsNET/1KM_NEW/WY2022'

##### Get list of all files within the specified YEAR (here, Option 2: Local Server Access was used)

In [52]:
#Retrive the list of files for the entered YEAR 

#Define an empty list to store the files based on User input YEAR
file_list = []

#Get the list of files for the specified year as file_list
for u in range(len(fews_wy_list)):
    if fews_wy_list[u] == year_fews:
        file_list = glob.glob(file_path + "**\*.nc")
        file_list
        file_list.sort()

#### Specify the PARAMETER whose values/times-series is required for analysis

In [39]:
#Selecting one parameter for testing
para_name = "SWE_inst"  #the chosen paramter is SWE: Snow Water Equivalent

# Other available parameters:
# dict_keys(['time', 'X', 'Y', 'lat', 'lon', 'Swnet_tavg', 
# 'Snowf_tavg', 'Rainf_tavg', 'Evap_tavg', 'Qs_tavg', 'Qsb_tavg', 
# 'Qsm_tavg', 'RadT_tavg', 'SWE_inst', 'SnowDepth_inst', 'SoilMoist_tavg', 
# 'SoilTemp_tavg', 'Snowcover_inst', 'Rainf_f_tavg', 'Tair_f_tavg', 
# 'Qair_f_tavg', 'Psurf_f_tavg', 'SWdown_f_tavg'])

### Geotransform parameters to calculate Zonal Statistics

- Get one single dataset to define the lat, long values (RASTER grid size)
- These grid sizes will be used to feed the geotransform parameters required to calculate statistics
- Here, Option 2: Using Local server access was used for geotransform process


#### Geotransform Option 1: Using TDS Catalog

In [ ]:
#NOTE: THE YEAR VALUES REQUIRES MANUAL UPDATES (TO CALCULATE IN YEARLY BASIS)
#Get one single dataset to define the lat, long, grid size, etc for geotransform parameters required to calculate statistics

# # STEP 1:
# #Select a single .nc dataset and apply remote access to read the single .nc file
# catalog = TDSCatalog(fews_2011)
# ds = catalog.datasets[dataset_2011_f[0]]
# dataset = ds.remote_access()

# # STEP 2:
# # Defining the latitude array (make sure variable name is lat
# lats = dataset.variables['Y'][:]  
# # Defining the longitude array (make sure variable name is lon)
# lons = dataset.variables['X'][:]

# #STEP 3: Grid size calculation
# delta_lat = np.abs(lats[1]-lats[0])
# delta_lon = np.abs(lons[1]-lons[0])

# #Step 4: Setting up GeoTransform variable
# geotransform = rio.transform.from_origin(lons.min(), lats.max(), delta_lat, delta_lon)

#### Geotransform Option 2: Using Local server access

In [40]:
#NOTE: THE YEAR VALUES REQUIRES MANUAL UPDATES (TO CALCULATE IN YEARLY BASIS)

# STEP 1:
#Take one nc file to extract lat long grid size for geotransform params
dataset = nc.Dataset(file_list[0], 'r')

# STEP 2:
# Defining the latitude array (make sure variable name is lat
lats = dataset.variables['Y'][:]  
# Defining the longitude array (make sure variable name is lon)
lons = dataset.variables['X'][:]

# STEP 3.1: To check image orientation using latitute values
delta_lat = lats[1]-lats[0]

#STEP 3.2: Grid size calculation
delta_lat_abs = np.abs(lats[1]-lats[0])
delta_lon_abs = np.abs(lons[1]-lons[0])

#Step 4: Setting up GeoTransform variable
geotransform = rio.transform.from_origin(lons.min(), lats.max(), delta_lat_abs, delta_lon_abs)

### Calculate Zonal Statistics for each polygon based on the input shapefile

In [ ]:
#Create an empty data series to store the Statistics of SWE using the imported SHP file

# Format: "SWE_year_indus = []"
SWE_2022_indus = []   

#NOTE: THE YEAR VALUES REQUIRES MANUAL UPDATES (TO CALCULATE IN YEARLY BASIS)

for index, rows in shp_indus.iterrows():
    #Assigning respective ID and other attribute from SHP file to add into dataframe
    pid = rows.ID   
    l1 = rows.L1_Name
    l2 = rows.L2_Name
    l3 = rows.L3_Name
    mc = rows.Mcode
    geo = rows.geometry
    
    for i in file_list:
        try:
            dataset = nc.Dataset(i, 'r')
            time = dataset.variables['time'][:]
            lis_var = dataset.variables
            field = dataset.variables[para_name][:]

            for timestep, v in enumerate(time):
                nc_arr = field[timestep]
                nc_arr2 = nc_arr.copy()
                nc_arr2[nc_arr2 < -9000] = np.nan  #comparator to drop nodata fills
                if delta_lat > 0:
                    nc_arr2 = nc_arr2[::-1]  #vertically flip the array to correct the orientation

                dt_str = nc.num2date(lis_var['time'][timestep], units=lis_var['time'].units,
                                          calendar=lis_var['time'].calendar)
                strTime = str(dt_str)
                dt_str = datetime.strptime(strTime, '%Y-%m-%d %H:%M:%S')
                new_dt = dt_str.strftime('%Y-%m-%d')

                #Calculating zonal statistics using RasterStats python library
                #Here, the statistics metrics include: min, max, mean, median, std and count
                
                #Using Rasterstats to calculate the statistics
                tt = rstats.zonal_stats(geo, nc_arr2, affine=geotransform, stats="min max mean median std count")
                lowest_value  = round(tt[0]["min"], 3)
                highest_value = round(tt[0]["max"], 3)
                avg_value = round(tt[0]["mean"], 3)
                med_value = round(tt[0]["median"], 3)
                stdev = round(tt[0]["std"], 3)
                count = round(tt[0]["count"], 3)

                ##############################################
                
                if avg_value:
                #Update the name according the User Input YEAR
                    SWE_2022_indus.append([pid, l1, l2, l3, mc, new_dt, lowest_value, highest_value, avg_value, med_value, stdev, count])
                else:
                    SWE_2022_indus.append([pid, l1, l2, l3, mc, new_dt, None, None, None, None, None, None])
                
                ##############################################
        except Exception as e:
            print(e)

In [ ]:
#Converting panda series to panda Dataframe and saves the dataset in CSV file

SWE_2022_indus_df = pd.DataFrame(SWE_2022_indus, columns = ['ID','L1_Name','L2_Name', 'L3_Name', 'Mcode', 'Date', 'Min', 'Max', 'Mean', 'Median', 'Std', 'Count'])
SWE_2022_indus_df.to_csv("SWE_2022_indus_statistics.csv", index=False, encoding='utf-8')

#NOTE: update the name of dataframe and output .csv file according to the input date